## 一个简单的查重测试

### 1. 基本数据清理

这两行是导入包和数据，直接用 pandas 导入 Excel 数据

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('/Users/Scott/Desktop/tst.xlsx')

用 info 方法简单查看数据基本情况，发现有 6 行的上线日期是空的，57 行的商品条码是空的，...

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 20 columns):
上线日期    272 non-null float64
众库编码    278 non-null object
一级类别    278 non-null object
商品条码    221 non-null float64
商品名称    256 non-null object
品牌      276 non-null object
原产地     276 non-null object
规格      238 non-null object
批发价     276 non-null float64
零售价     276 non-null object
电商价     276 non-null float64
返库币     276 non-null float64
返库币值    215 non-null float64
天猫销量    274 non-null object
京东销量    247 non-null float64
价格星级    0 non-null float64
人员      116 non-null object
项目经理    276 non-null object
库存      274 non-null object
备注      213 non-null object
dtypes: float64(8), object(12)
memory usage: 43.5+ KB


众库编码是最为重要的，首先看看具体的众库编码有没重复

In [4]:
df1 = df[df['众库编码'].duplicated(keep=False)]
df1

,上线日期,众库编码,一级类别,商品条码,商品名称,品牌,原产地,规格,批发价,零售价,电商价,返库币,返库币值,天猫销量,京东销量,价格星级,人员,项目经理,库存,备注
64,NaN,H59200042-H59200050,空置,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,H59200042-H59200050,空置,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


发现有两行重复数据，且大部分列是空值，drop 删除

In [5]:
df = df.drop([64,65])

再查看基本信息，发现数据已经减少两行了

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276 entries, 0 to 277
Data columns (total 20 columns):
上线日期    272 non-null float64
众库编码    276 non-null object
一级类别    276 non-null object
商品条码    221 non-null float64
商品名称    256 non-null object
品牌      276 non-null object
原产地     276 non-null object
规格      238 non-null object
批发价     276 non-null float64
零售价     276 non-null object
电商价     276 non-null float64
返库币     276 non-null float64
返库币值    215 non-null float64
天猫销量    274 non-null object
京东销量    247 non-null float64
价格星级    0 non-null float64
人员      116 non-null object
项目经理    276 non-null object
库存      274 non-null object
备注      213 non-null object
dtypes: float64(8), object(12)
memory usage: 45.3+ KB


### 2. 深度查重

In [7]:
df['商品名称'].head()

0         R10U
1         R12U
2         R18T
3    R1000TC北美
4      R1700BT
Name: 商品名称, dtype: object

从上方这头几行可以看到，商品名称也有特定的型号，所以也可以查重看看

In [8]:
df[df['商品名称'].duplicated()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 19 to 240
Data columns (total 20 columns):
上线日期    25 non-null float64
众库编码    25 non-null object
一级类别    25 non-null object
商品条码    21 non-null float64
商品名称    6 non-null object
品牌      25 non-null object
原产地     25 non-null object
规格      6 non-null object
批发价     25 non-null float64
零售价     25 non-null object
电商价     25 non-null float64
返库币     25 non-null float64
返库币值    23 non-null float64
天猫销量    25 non-null object
京东销量    25 non-null float64
价格星级    0 non-null float64
人员      2 non-null object
项目经理    25 non-null object
库存      25 non-null object
备注      23 non-null object
dtypes: float64(8), object(12)
memory usage: 4.1+ KB


重复率居然高达 27 个，非常奇怪，仔细查看下数据

In [9]:
df[df['商品名称'].duplicated(keep=False)].iloc[:, :10].head()

,上线日期,众库编码,一级类别,商品条码,商品名称,品牌,原产地,规格,批发价,零售价
18,827.0,F59200052-1,数码电器,NaN,SoundTouch 10,Bose,墨西哥/马来西亚,白,1500.0,1700
19,827.0,F59200052-2,数码电器,NaN,SoundTouch 10,Bose,墨西哥/马来西亚,黑,1500.0,1700
20,827.0,F59200053-1,数码电器,NaN,SoundTouch 20 III,Bose,墨西哥/马来西亚,白,2692.0,3200
21,827.0,F59200053-2,数码电器,NaN,SoundTouch 20 III,Bose,墨西哥/马来西亚,黑,2692.0,3200
23,827.0,F59200055-1,数码电器,NaN,SoundTouch 30 III,Bose,墨西哥/马来西亚,黑,4024.0,4580


从上方的输出发现原来商品名称一样的产品，规格不一样，这个解决思路也很简单，可以把商品名称和规格拼接起来单独做一列，针对这列查重即可

In [10]:
df['Added']  = df['商品名称'] + ' ' + df['规格']

In [11]:
df.Added.head(20)

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
5                 NaN
6                 NaN
7                 NaN
8                 NaN
9                 NaN
10                NaN
11                NaN
12                NaN
13                NaN
14                NaN
15                NaN
16                NaN
17                NaN
18    SoundTouch 10 白
19    SoundTouch 10 黑
Name: Added, dtype: object

In [12]:
df[df.Added.duplicated() & df.Added.notnull()]

,上线日期,众库编码,一级类别,商品条码,商品名称,品牌,原产地,规格,批发价,零售价,...,返库币,返库币值,天猫销量,京东销量,价格星级,人员,项目经理,库存,备注,Added


上方返回数据为空，所以此表基本没有重复了。把清理好的数据用 pd.to_excel 方法导出即可。

**备注：** 写于 9.3  日早，15 分钟完成

**建议：** 查重 Excel 解决不了根本问题，手工输难免出错。建议公司招聘 DBA（数据库管理员）或运维，让数据人员把清理好的数据一键导入即可。省时省力出错少。